# Visualization Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [3]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [4]:
_ = main_flow(max_weeks=3, max_games=3, max_plays=3)

16:56:52.928 | INFO    | prefect.engine - Created flow run 'astute-caracara' for flow 'main-flow'

16:56:53.108 | INFO    | Flow run 'astute-caracara' - Created task run 'read_csv-3609c996-0' for task 'read_csv'

16:56:53.110 | INFO    | Flow run 'astute-caracara' - Executing 'read_csv-3609c996-0' immediately...

16:56:53.419 | INFO    | Task run 'read_csv-3609c996-0' - Finished in state Completed()

16:56:53.446 | INFO    | Flow run 'astute-caracara' - Created task run 'read_csv-3609c996-1' for task 'read_csv'

...

16:57:07.042 | INFO    | Task run 'write_csv-386fe2af-4' - Finished in state Completed()

16:57:07.068 | INFO    | Flow run 'astute-caracara' - Created task run 'write_csv-386fe2af-5' for task 'write_csv'

16:57:07.070 | INFO    | Flow run 'astute-caracara' - Executing 'write_csv-386fe2af-5' immediately...

16:57:07.137 | INFO    | Task run 'write_csv-386fe2af-5' - Finished in state Completed()

16:57:07.185 | INFO    | Flow run 'astute-caracara' - Finished in state Completed('All states completed.')

In [5]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [6]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1, 2, 3), value=1), Dropdown(description='Game ID'…